# Tutorial of OPFIR package

The OPFIR package simulates the THz OPFIR laser system, for both prolate symmetric top molecules such as CH$_3$F (currently in the master branch), and linear molecules such as N$_2$O (currently in the N2O_VV branch). 
The following disucssion focus on the N$_2$O modeling. 

In the model, first set up the parameters of the problem by 
<pre><code>
OPFIR.N2O(...)
</code></pre>
There are many parameters in the setting, following lists the most important ones:
<pre><code>
    radius = 0.25, # in cm
    L = 15, # in cm
    T = 300,
    pressure = 100.0,
    power = 10.0,
    ####################################
    ## molecule setup
    ####################################
    M = 44,
    σ_DD = 35,
    σ_GKC = 50,
    σ_SPT = 50,
    ####################################
    ## pump setup
    ####################################
    JL = 11,
    pumpbranch = "R",
    f_offset = 0.e6,
    n_rot = 18,
    n0 = 1.0,
    ####################################
    ## model/solver setup
    ####################################
    num_layers = 8,
    solstart_flag = 0,
    optcavity = false,
    D_factor = 1.0,
    WiU = 0,
    WiL = 0,
    n_vib = 1,
    pbfactor = 1.0, # pressure broadening factor
    qclbroadening = 0. # qcl broadening width, in Hz
    </code></pre>
For example,
<pre><code>
p = OPFIR.N2O(pressure=20, power=0.25, num_layers=6, JL=14, n_vib=10,
              σ_DD=35, σ_GKC=15, σ_SPT=15, qclbroadening=1.e6)
</code></pre>
sets up a simulation with pressure 20mTorr, pump power 250mW, 6 discretization in radial direction, JL=14, including 10 vibrational levels, and a few collisional cross sections values in $A^2$.

In addition, one needs to set the cavity mode, lasing level ('U', or 'L'), mirror transmission and reflection coefficients in other places. It's better to include them in the parameter setting. I'll fix it recently.

<code>p, sol = OPFIR.func(p)</code> computes the non-thermal population in <code>sol</code> and updates absorption coefficient in <code>p.alpha_r[1]</code> self-consistently. The length of <code>sol</code> is <code>p.num_layers * p.layer_unknown + p.n_vib</code> in which <code>p.layer_unknown = p.n_rot * p.num_freq + p.n_vib</code> is the number of unknowns in one radial "layer". 

<code>laspower, sol, p, taus = OPFIR.outputpower(p, level, cavitymode; mumps_solver=0, lossfactor=1.0)</code> computes the output power, which calls <code>func(p)</code> to compute the population inversion. In addition, it computes the saturation time $\tau_s$ numerically. $\tau_s$ is the 4th output from the function. And lastly, it calculates the output power by matching gain and cavity loss. Note that the absorption coefficient of output <code>p</code> is updated.

<code>mumps_solver</code> specifies different linear solver for sparse matrix, default value is 0, meaning from Julia itself.

<code>lossfactor</code> is a multiplication factor for cavity loss to fit the cavity loss. So in the implementation, the real cavity loss is set as <code>lossfactor * OPFIR.cavityloss(p, level, cavitymode)</code>.

Here, <code>level</code> can be either <code>'U'</code> or <code>'L'</code> for direct lasing and refilling lasing. <code>cavitymode</code> specifies the cavity mode, e.g. <code>"TE01"</code>. 

About cavity loss, <code>OPFIR.cavityloss(p, level, cavitymode)</code> calculates the total cavity loss, including transmission loss and ohmic loss <code>OPFIR.ohmicloss(p, level, cavitymode)</code>.

Other useful functions:

<code>OPFIR.pumpthreshold(p, level, cavitymode; mumps_solver=0, lossfactor=1.0, guess=0.1)</code> computes the pump threshold power by solving <code>OPFIR.outputpower=0</code>, with options of <code>lossfactor</code>.

<code>OPFIR.totinv(p, sol, level)</code> computes the spatial averaged population inversion for transition U or L with obtained population from <code>OPFIR.func</code>

### Examples:

In [ ]:
using Pkg
Pkg.activate("..")
using OPFIR

In [ ]:
p = OPFIR.N2O(pressure=20, power=0.25, num_layers=6, JL=14, n_vib=10,σ_DD=35, σ_GKC=15, σ_SPT=15,qclbroadening=1.e6)
level = "U"
cavitymode = "TE01"
lossfactor = 0.3/OPFIR.cavityloss(p,level,cavitymode)

In [ ]:
# OPFIR.pumpthreshold(p, level, cavitymode, mumps_solver=0, lossfactor=lossfactor, guess=0.04)
laspower, sol, p, taus = OPFIR.outputpower(p, level, cavitymode; mumps_solver=0, lossfactor=1.0)

In [ ]:
using PyPlot
figure(figsize=(4,3))
THzpower = []
for Rcell in [0.2, 0.4, 0.8]
    THzpower = []
    for pressure in 0.001:2:100
        p = OPFIR.N2O(pressure=pressure, JL=26, radius=Rcell)
        push!(THzpower, OPFIR.PTHz(0.25, p, model="FLM"))
    end
    plot(2:2:100, 1000THzpower)
end
ylim(0,)
xlim(0,)
xlabel("pressure (mTorr)")
ylabel("THz power (mW)")
legend([L"R_\mathrm{cell}=2 \mathrm{mm}", L"R_\mathrm{cell}=4 \mathrm{mm}", L"R_\mathrm{cell}=8 \mathrm{mm}"])